In [1]:
import os
import sys
import abatement_functions as af
import DB2Gams

# **1: Final goods sector**

This runs the final goods sector alone, to provide some understanding of relevant classes in the abatement model.

#### **1.1: Define nesting structure and read in data via nesting_trees**

Define nesting structure of inputs by mapping: Keys are aggregates, values are inputs in that aggregate. 

In [2]:
FG_tree = af.nesting_tree(
    name='FG',
    tree = {
        'KE': ['K','E'],
        'E' : ['E1','E2']},
    **{'setname': 'n',
       'alias': 'nn',
       'alias2' : 'nnn',
       'inpname': 'in_FG',
       'aggname': 'agg_FG',
       'mapname': 'in_FG2agg_FG',
       'outname': 'out_FG',
       'sector':  'sec_FG'}
)

Create sets, subsets, and mappings from tree:

In [3]:
FG_tree.run_all()

Read in data from excel:

In [4]:
read_type = {'1dvars': ['sigma'], # list sheets with with data of variables defined over 1 set
             'vars_panel': {'mu': 2}} # dictionary with variables defined over n sets written in panel-data form. Key refers to sheet name, value refers to number of sets.
FG_tree.database.read_from_excel('Make_data//FinalGoods.xlsx',read_type)

#### **1.2: Initialize abatement_model with nesting_tree**

In [5]:
FG = af.abatement_model(FG_tree)
FG.run_abatement_model(repo=os.getcwd()+'\\gms\\FG',type_='CES',export_settings=True)

#### **1.3: Finally, use the model to initialize gams model**

Initialize model with database, and workfolder:

In [6]:
FG_model = DB2Gams.gams_model(os.getcwd()+'\\workfolder')

In [7]:
FG.model.__dict__

{'name': 'FG',
 'placeholders': {'FG': 'FG'},
 'databases': {'FG': <DataBase.py_db at 0x253d50b0ec8>},
 'run_file': None,
 'blocks': ['M_FG'],
 'g_endo': ['FG_endo'],
 'g_exo': ['FG_tech', 'FG_exo'],
 'solve': None,
 'files': {'FG_functions.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\gms\\FG',
  'FG_sets.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\gms\\FG',
  'FG_parameters.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\gms\\FG',
  'FG_groups.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\gms\\FG',
  'FG_blocks.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\gms\\FG',
  'FG_CollectFile.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\Abatement_project\\InputDisplacingAbatement\\gms\\FG'},
 'collect_file': None,
 'collect_files': ['FG_CollectFile.gms'],
 'root_file': None,
 

*NB: We can pass options to the solver using dict inputs*

In [8]:
# FG_model.run(FG.model,options_run={'output': sys.stdout}) # Run with gams output in cell
FG_model.run(FG.model) # run without

The database:

## **2: Energy-Service producing firms**

In [9]:
ES_tree = af.nesting_tree(
    name='ES',
    tree= {
        'E1': ['E10','E11'],
        'E2': ['E20','E21','E22']},
    **{'setname': 'n',
       'alias'  : 'nn',
       'alias2' : 'nnn',
       'inpname': 'in_ES',
       'aggname': 'agg_ES',
       'mapname': 'in_ES2agg_ES',
       'outname': 'out_ES',
       'sector': 'sec_ES'}
)

Create sets, read in data from excel:

In [10]:
ES_tree.run_all()
read_type = {'1dvars': ['sigma'], 'vars_panel': {'mu': 2}}
ES_tree.database.read_from_excel('Make_data//ES.xlsx',read_type)

Initialize as model:

In [11]:
ES = af.abatement_model(ES_tree)
ES.run_abatement_model(repo=os.getcwd()+'\\gms\\ES',type_='normalized_CES',export_settings=True)

Run model:

In [12]:
gams_ES = DB2Gams.gams_model(work_folder = os.getcwd()+'\\workfolder')

In [13]:
gams_ES.run(ES.model)

## **3: Energy-Service-component producing firms**

In [14]:
ESC_tree = af.nesting_tree(
    name='ESC',
    tree= {
        'E10': ['E100'],
        'E11': ['E110','E111'],
        'E20': ['E200'],
        'E21': ['E210','E211'],
        'E22': ['E220','E221','E222']},
    **{'setname': 'n',
       'alias'  : 'nn',
       'alias2' : 'nnn',
       'inpname': 'in_ESC',
       'aggname': 'agg_ESC',
       'mapname': 'in_ESC2agg_ESC',
       'outname': 'out_ESC',
       'sector': 'sec_ESC'}
)

In [15]:
ESC_tree.run_all()
read_type = {'1dvars': ['sigma']}
ESC_tree.database.read_from_excel('Make_data//ESC.xlsx',read_type)

In [16]:
ESC = af.abatement_model(ESC_tree)
ESC.run_abatement_model(repo=os.getcwd()+'\\gms\\ES',type_='MNL',export_settings=True)

Run model:

In [17]:
gams_ESC = DB2Gams.gams_model(work_folder = os.getcwd()+'\\workfolder')

In [18]:
gams_ESC.run(ESC.model)

## **4: Energy-service technologies**

A nesting tree with input-split, and output-split models:

In [19]:
Tech_tree = af.nesting_tree_with_CET(
    name='Tech',
    tree= {'in' : {'Y': ['E1','K']},
           'out': {'Y': ['Y1','Y2'],
                   'E': ['E1','E2']}}
)

In [20]:
Tech_tree.run_all()

In [21]:
Tech = af.abatement_model_with_CET(Tech_tree)
Tech.run_abatement_model(repo=os.getcwd()+'\\gms\\Tech',type_in='CES',type_out='CES',export_settings=True)

In [22]:
gams_Tech = DB2Gams.gams_model(work_folder = os.getcwd()+'\\workfolder')

In [23]:
gams_Tech.run(Tech.model)